# Azure DALL·E 画像生成の例

> 注意: openaiライブラリの新しいバージョンが利用可能です。https://github.com/openai/openai-python/discussions/742 を参照してください

このノートブックでは、Azure OpenAIサービスを使用して画像を生成する方法を示します。

## セットアップ

まず、必要な依存関係をインストールします。

In [ ]:
! pip install "openai>=0.28.1,<1.0.0"
# We need requests to retrieve the generated image
! pip install requests
# We use Pillow to display the generated image
! pip install pillow 
# (Optional) If you want to use Microsoft Active Directory
! pip install azure-identity

In [ ]:
import os
import openai

さらに、Azure OpenAI Serviceに適切にアクセスするために、[Azure Portal](https://portal.azure.com)で適切なリソースを作成する必要があります（詳細な手順については[Microsoft Docs](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)で確認できます）。

リソースが作成されたら、最初に必要なのはそのエンドポイントです。エンドポイントは、*「リソース管理」*セクション内の*「キーとエンドポイント」*セクションで確認できます。この情報を使用して、SDKをセットアップします：

In [ ]:
openai.api_base = '' # Add your endpoint here

# At the moment DALL·E is only supported by the 2023-06-01-preview API version
openai.api_version = '2023-06-01-preview'

### 認証

Azure OpenAI サービスは、APIキーとAzure認証情報を含む複数の認証メカニズムをサポートしています。

In [ ]:
use_azure_active_directory = False

#### APIキーを使用した認証

OpenAI SDKを*Azure APIキー*を使用するように設定するには、`api_type`を`azure`に設定し、`api_key`をエンドポイントに関連付けられたキーに設定する必要があります（このキーは[Azure Portal](https://portal.azure.com)の*「リソース管理」*の下にある*「キーとエンドポイント」*で確認できます）

In [ ]:
if not use_azure_active_directory:
    openai.api_type = 'azure'
    openai.api_key = os.environ["OPENAI_API_KEY"]

> 注意: この例では、コード内で変数を設定することでライブラリがAzure APIを使用するように設定しました。開発時には、代わりに環境変数を設定することを検討してください：

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```

#### Microsoft Active Directoryを使用した認証
それでは、Microsoft Active Directory認証を通じてキーを取得する方法を見てみましょう。

In [ ]:
from azure.identity import DefaultAzureCredential

if use_azure_active_directory:
    default_credential = DefaultAzureCredential()
    token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

    openai.api_type = 'azure_ad'
    openai.api_key = token.token

トークンは一定期間有効で、その後期限切れになります。すべてのリクエストで有効なトークンが送信されるようにするために、`requests.auth`にフックすることで期限切れのトークンを更新できます：

In [ ]:
import typing
import time
import requests

if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

if use_azure_active_directory:
    session = requests.Session()
    session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

    openai.requestssession = session

## 生成

セットアップと認証が完了したので、Azure OpenAIサービスで画像を生成し、返されたURLから画像を取得できるようになりました。

#### 1. 画像を生成する

このプロセスの最初のステップは、実際に画像を生成することです：

In [ ]:
generation_response = openai.Image.create(
    prompt='A cyberpunk monkey hacker dreaming of a beautiful bunch of bananas, digital art',
    size='1024x1024',
    n=2
)

print(generation_response)

`Image.create`の呼び出しからのレスポンスを受け取った後、`requests`を使用してURLからダウンロードします。

In [ ]:
import os
import requests

# First a little setup
image_dir = os.path.join(os.curdir, 'images')
# If the directory doesn't exist, create it
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)

# With the directory in place, we can initialize the image path (note that filetype should be png)
image_path = os.path.join(image_dir, 'generated_image.png')

# Now we can retrieve the generated image
image_url = generation_response["data"][0]["url"]  # extract image URL from response
generated_image = requests.get(image_url).content  # download the image
with open(image_path, "wb") as image_file:
    image_file.write(generated_image)

画像をダウンロードした後、[Pillow](https://pypi.org/project/Pillow/)ライブラリを使用して画像を開いて表示します：

In [ ]:
from PIL import Image 

display(Image.open(image_path))